In [1]:
import datasketch as ds
import struct
import pickle
import base64
import time
import random
import pickle

In [2]:
import datasketch as ds
import hbase_connector as hb

> Note: ds.lsh.MinHashLSH(...) may take some time to execute the first time, as it's creating tables inside hbase

## LSH with HBase as backing memory

In [3]:
pool = hb.HbaseConnection(host="datanode1", port=9090)
lsh = ds.lsh.MinHashLSH(threshold=0.5, storage_config={'type': 'hbase', 'basename': b'demotablelsh_v4', 'hbase_pool': pool}, prepickle=True)

In [4]:
def create_hash(string):
    mh2 = ds.MinHash()
    for c in string:
        mh2.update(c.encode('utf8'))
    return mh2

In [5]:
values = ['hello', 'hotel', 'hamburger', 'house', 'ham', 'oreo', 'chocolate', 'silk', 'rodeo'] 
hashes = [create_hash(v) for v in values]
for key, h in zip(values, hashes):
    lsh.insert(key, h)

## LSH with local memory for a comparison

In [6]:
lsh_d = ds.lsh.MinHashLSH(threshold=0.5)
for key, h in zip(values, hashes):
    lsh_d.insert(key, h)

## And now for a query

In [7]:
query = create_hash("hotel")
# Hbase, Local
lsh.query(query), lsh_d.query(query)
# They match (except order) :)

(['chocolate', 'hotel', 'hello'], ['hello', 'chocolate', 'hotel'])